## 8.3 Object Counting Faster R-CNN

### 8.3.1 Install Supervision
<img src="resource/rf-supervision-banner.png" width="700px"><br><br>
>.
>
> ⚠️ <font color="orange">If you already create conda environment 'BelajarSuperVision', you can skip the following step then jump to part #8.3.2, </font><br>
>
>.
- Since Supervision working in `python3.8`, we need to create new conda environment with name `BelajarSuperVision` using that python version
- Open `Anaconda prompt`
- create new environment `BelajarSuperVision`
    ```
    conda create --name BelajarSuperVision python=3.8
    ```
- activate environment
    ```
    conda activate BelajarSuperVision
    ```
- run to install supervision & ultralytics
    ```
    pip install ipykernel
    pip install supervision
    pip install ultralytics
    pip install onnx --user
    pip install onnxruntime
    ```
- Close VS Code, then reopen it
- Open `6.2 supervision-object-counting.ipynb`
- Choose `BelajarSuperVision` as python environment<br>
<img src="resource/sv-image.png" width="300px">

<br><br><br><br>
### 8.3.2 Object Counting Faster R-CNN using ONNX Runtiome + SuperVision
- import library

In [ ]:
import cv2
import numpy as np
import torch
import onnx
import onnxruntime as ort
import supervision as sv
import utils

utility = utils.Utils()

- Define class labels

In [ ]:
CLASS_LABELS = ["background", "scissors"]  # Modify according to your dataset

- load sample image

In [ ]:
original_image = cv2.imread("image1.jpg")

if(original_image.shape[1] > 900) : 
    original_image = cv2.resize(original_image, (0, 0), fx=0.5,fy=0.5) # resize 50% if image to big

- Load Faster R-CNN ONNX Model into ONNX Runtime

In [ ]:
# Load the ONNX model
MODEL_NAME = "model/fasterrcnn_resnet50_fpn_v2_scissors.onnx"
onnx_model = onnx.load(MODEL_NAME)
onnx.checker.check_model(onnx_model)

# Initialize inference with ONNX Runtime
ort_session = ort.InferenceSession(MODEL_NAME)

- run inference on input tensor to ONNX Faster R-CNN model

In [ ]:
# Preprocess the frame for Faster R-CNN
image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))
image_norm = (image / 255.0).astype(np.float32)
image_tensor = np.transpose(image_norm, (2, 0, 1))
image_tensor = np.expand_dims(image_tensor, axis=0) # Shape: [1, C, H, W]

# run inference
outputs = ort_session.run(None, {"input": image_tensor})
print("ONNX model outputs:", outputs[0])

- Define polygon using mouse click with `cv2.selectROI`

In [ ]:
# Set up the polygon using mouse click
x, y, w, h = cv2.selectROI("Select ROI", original_image, fromCenter=False, showCrosshair=False)

polygon = np.array([
    [x, y],           # Top-left corner
    [x + w, y],       # Top-right corner
    [x + w, y + h],   # Bottom-right corner
    [x, y + h]        # Bottom-left corner
])
cv2.destroyAllWindows()
print(polygon)

- or, <font color="orange">Define polygon zone manually</font> here,
    - <i>uncomment below code to use it!</i>

In [ ]:
# paste the polygon point here!
# polygon = np.array([[ 63,  61], [539,  61], [539, 601], [ 63, 601]]) # CHANGE TO YOUR OWN POLYGON

- Instantiate SuperVision Polygone Zone

In [ ]:
# Create Supervision's PolygonZone for object counting in the specified polygon
zone = sv.PolygonZone(polygon=polygon)

# Create Supervision annotators for boxes, labels, and the polygon zone
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.GREEN, thickness=2)

- Run object counting

In [ ]:
# Filter results by confidence score (e.g., >0.5)
threshold = 0.89
mask = outputs[2] > threshold # filter mask by Confidence scores > threshold
boxes = outputs[0][mask] # Bounding boxes
labels = outputs[1][mask] # Class labels
scores = outputs[2][mask] # Confidence scores

# Rescale detected box to original image size
boxes = utility.rescale_box(boxes, original_image, input_size=[224,224])


# Convert predictions to Supervision's Detections format
detections = sv.Detections(
    xyxy=boxes,        # Use boxes directly as xyxy format (xmin, ymin, xmax, ymax)
    confidence=scores, # Confidence scores for each detection
    class_id=labels    # Class IDs for each detection
)


# Trigger the zone counting for the filtered detections
zone.trigger(detections=detections)


# Annotate bounding boxes and labels
box_labels = [
    f"{CLASS_LABELS[class_id]} {confidence:.2f}"
    for class_id, confidence in zip(detections.class_id, detections.confidence)
]
result_image = original_image.copy()
result_image = box_annotator.annotate(scene=result_image, detections=detections)
result_image = label_annotator.annotate(scene=result_image, detections=detections, labels=box_labels)


# Annotate the polygon zone and display the count
object_count_label = f"count {zone_annotator.zone.current_count}"
result_image = zone_annotator.annotate(scene=result_image, label=object_count_label)



# Show the result
cv2.imshow('Frame', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
